In [3]:
import argparse
from glob import glob

import numpy as np
from PIL import Image
from keras import backend as K
from keras import losses
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Input, MaxPooling2D
from keras.layers import concatenate, Conv2D, Conv2DTranspose, Dropout, ReLU
from keras.models import Model
from keras.optimizers import Adam
from numpy import random
import tensorflow as tf
from random import randint
import keras
from keras_preprocessing import image
import cv2
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [0]:
batch_size = 16
input_shape = (64, 64)

def get_unet(do=0, activation=ReLU):
    inputs = Input((None, None, 3))
    conv1 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(inputs)))
    conv1 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(conv1)))
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(pool1)))
    conv2 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(conv2)))
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(pool2)))
    conv3 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(conv3)))
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(pool3)))
    conv4 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(conv4)))
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Dropout(do)(activation()(Conv2D(512, (3, 3), padding='same')(pool4)))
    conv5 = Dropout(do)(activation()(Conv2D(512, (3, 3), padding='same')(conv5)))

    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    conv6 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(up6)))
    conv6 = Dropout(do)(activation()(Conv2D(256, (3, 3), padding='same')(conv6)))

    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    conv7 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(up7)))
    conv7 = Dropout(do)(activation()(Conv2D(128, (3, 3), padding='same')(conv7)))

    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    conv8 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(up8)))
    conv8 = Dropout(do)(activation()(Conv2D(64, (3, 3), padding='same')(conv8)))

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    conv9 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(up9)))
    conv9 = Dropout(do)(activation()(Conv2D(32, (3, 3), padding='same')(conv9)))

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])

    model.compile(optimizer=Adam(lr=1e-3), loss=losses.binary_crossentropy, metrics=['accuracy'])


    return model


def read_input(path):
    x = np.array(Image.open(path))/255.
    return x


def read_gt(path):
    x = np.array(Image.open(path))/255.
    return x[..., np.newaxis]


def random_crop(img, mask, crop_size=input_shape[0]):
    imgheight= img.shape[0]
    imgwidth = img.shape[1]
    i = randint(0, imgheight-crop_size)
    j = randint(0, imgwidth-crop_size)

    return img[i:(i+crop_size), j:(j+crop_size), :], mask[i:(i+crop_size), j:(j+crop_size)]


def gen(data):
    while True:
        repeat = 4
        index= random.choice(list(range(len(data))), batch_size//repeat)
        index = list(map(int, index))
        list_images_base = [read_input(data[i][0]) for i in index]
        list_gt_base = [read_gt(data[i][1]) for i in index]

        list_images = []
        list_gt = []

        for image, gt in zip(list_images_base, list_gt_base):

            for _ in range(repeat):
                image_, gt_ = random_crop(image.copy(), gt.copy())
                list_images.append(image_)
                list_gt.append(gt_)

        yield np.array(list_images), np.array(list_gt)


In [8]:
data = list(zip(sorted(glob('DRIVE/training/images/*.tif')),
                      sorted(glob('DRIVE/training/1st_manual/*.gif'))))


repeat = 4
index= random.choice(list(range(len(data))), batch_size//repeat)
index = list(map(int, index))
list_images_base = [read_input(data[i][0]) for i in index]
list_gt_base = [read_gt(data[i][1]) for i in index]

list_images = []
list_gt = []

for image, gt in zip(list_images_base, list_gt_base):

    for _ in range(repeat):
        image_, gt_ = random_crop(image.copy(), gt.copy())
        list_images.append(image_)
        list_gt.append(gt_)

list_images = np.array(list_images)
list_gt = np.array(list_gt)
len(data)

20

In [15]:
activation = globals()["ReLU"]
dropout = 0.1

train_data = list(zip(sorted(glob('DRIVE/training/images/*.tif')),
                      sorted(glob('DRIVE/training/1st_manual/*.gif'))))

model = get_unet(do=dropout, activation=activation)

file_path = "weights.best.hdf5"

#try:
#    model.load_weights(file_path, by_name=True)
#except:
#    pass


#history = model.fit_generator(gen(train_data, au=True), epochs=100, verbose=2,
#                     steps_per_epoch= 100*len(train_data)//batch_size,
#                              use_multiprocessing=True, workers=16)

history = model.fit_generator(gen(train_data[:18]), validation_data=gen(train_data[18:20]), 
                              validation_steps=100*2//batch_size, epochs=5, 
                              steps_per_epoch= 100*len(train_data)//batch_size)

#model.save_weights(file_path)

Epoch 1/5
125/125 [==============================] - 29s 230ms/step - loss: 0.3500 - acc: 0.8940 - val_loss: 0.3014 - val_acc: 0.9073
Epoch 2/5
125/125 [==============================] - 22s 177ms/step - loss: 0.3201 - acc: 0.8905 - val_loss: 0.2982 - val_acc: 0.8989
Epoch 3/5
125/125 [==============================] - 23s 187ms/step - loss: 0.3155 - acc: 0.8889 - val_loss: 0.2765 - val_acc: 0.8983
Epoch 4/5
125/125 [==============================] - 23s 182ms/step - loss: 0.2503 - acc: 0.9121 - val_loss: 0.1516 - val_acc: 0.9459
Epoch 5/5
125/125 [==============================] - 22s 179ms/step - loss: 0.1807 - acc: 0.9357 - val_loss: 0.1352 - val_acc: 0.9516


In [0]:
!unzip DRIVE.zip